<a href="https://colab.research.google.com/github/dogang97/do/blob/main/JAX_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

산업수학 기반 데이터 분석 프로그램

김병천 (wizardbc@gmail.com)<br>
인하대학교 응용수학연구소 / 스마트소셜 AI 연구소

2022년 8월 17일 (수)<br>
더케이호텔 서울, 금강A홀

# Artificial Neural Network (ANN)

![ANN](https://upload.wikimedia.org/wikipedia/commons/3/3d/Neural_network.svg)

[Wikimedia](https://commons.wikimedia.org/wiki/File:Neural_network.svg)

* Each circle represents a real number.
* The input neurons (the green circles) represent a vector $(x_1, x_2)^T\in\mathbb R^2$.
* The edges from the input neurons to the hidden neurons (the blue circles) represent weights $\left(a_{ij}^{(1)}\right)$ and biases $\left(b_{ij}^{(1)}\right)$ in $\mathbb{R}^{5\times 2}$.
* The hidden neurons (the blue circles) represent a vector $\left(\sigma(y_1), \ldots, \sigma(y_5)\right)^T\in\mathbb R^5$ where $y_i = \sum_{j=1}^2\left(a_{ij}^{(1)}x_j+b_{ij}^{(1)}\right)$ and $\sigma:\mathbb R\rightarrow\mathbb R$ is an activation function.
* The edges from the hidden neurons to output neurons (the yello circle) represent weights $\left(a_{ij}^{(2)}\right)$ and biases $\left(b_{ij}^{(2)}\right)$ in $\mathbb{R}^{1\times 5}$.
* The output neurons represent a real number $\sum_{j=1}^5 \left(a_{ij}^{(2)}\sigma(y_i)+b_{ij}^{(2)}\right)$.


## Affine Maps

If $x=(x_1,\ldots, x_d)^T\in\mathbb R^d$, an affine map $W$ consists of linear map (matrix) $A=\left(a_{ij}\right)\in\mathbb R^{n\times d}$ (weights) and vector $b=(b_1,\ldots,b_n)^T\in\mathbb R^n$ (biases). Then $W(x)$ means $n$-dimensional vector $Ax+b=\left(\sum_{j=1}^d a_{ij}x_j+b_i\right)^T\in\mathbb R^n$.

## Component-wise Composition

Given a function $\sigma:\mathbb R\rightarrow\mathbb R$, and $y=\left(y_1,\ldots,y_n\right)^T\in\mathbb R^n$, $\sigma(y)$ means $\left(\sigma(y_1),\ldots,\sigma(y_n)\right)^T\in\mathbb R^n$.

Now we can write the neural network in the picture by
$$W_2\cdot\sigma\circ W_1$$
where
*  $W_1 = \left(A_1, b_1\right)$ with $A_1=\left(a_{ij}^{(1)}\right)\in\mathbb R^{5\times 2}$ and $b_1=\left(\sum_{j=1}^2 b_{ij}^{(1)}\right)^T\in\mathbb R^5$,
*  $W_2 = \left(A_2, b_2\right)$ with $A_2=\left(a_{ij}^{(2)}\right)\in\mathbb R^{1\times 5}$ and $b_2=\sum_{j=1}^5 b_{1j}^{(2)}\in\mathbb R^1$.

# Universal Approximation Theorem

## Arbitrary-width case

[Wikipedia](https://en.wikipedia.org/wiki/Universal_approximation_theorem#Arbitrary-width_case)

(1989, George Cybenko)

Fix a continuous function $\sigma:\mathbb R\rightarrow\mathbb R$ (activation function) and positive integers $d, D$. The function $\sigma$ is not a polynomial if and only if, for every continuous function $f:\mathbb R^d\rightarrow\mathbb R^D$ (target function), every compact subset $K$ of $\mathbb R^d$, and every $\epsilon > 0$ there exists a continuous function $f_\epsilon:\mathbb R^d\rightarrow\mathbb R^D$ with representation $$f_\epsilon = W_2\cdot\sigma\circ W_1,$$
where $W_1, W_2$ are composable affine maps and $\circ$ denotes component-wise composition, such that the approximation bound $$\sup_{x\in K}\|f(x)-f_\epsilon(x)\|<\epsilon.$$

### Sketch
Given continuous $\sigma:\mathbb R\rightarrow\mathbb R$, positive integers $d, D$.

$\sigma$ is not a polynomial $\iff$
* $\forall f:\mathbb R^d\rightarrow \mathbb R^D$,
* $\forall$ compact $K\subset \mathbb R^d$,
* $\forall \epsilon>0$,

there exists a neural network $\hat f$:
* $d$ input neurons,
* $D$ output neurons,
* only one hidden layer with an arbitrary number of hidden neurons having activation function $\sigma$,

such that
$$\sup_{x\in K}\|f(x)-\hat f(x)\|<\epsilon.$$

#### Unit Step Function (Heaviside Step Function)

$$H(x) := \begin{cases} 1, & x > 0 \\ 0, & x \le 0 \end{cases}$$
Sometimes,
$$H(x) := \begin{cases} 1, & x > 0 \\ \frac{1}{2}, & x = 0 \\ 0, & x < 0 \end{cases}$$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('dark_background')

# H = np.vectorize(lambda x: 1 if x > 0.0 else 0.0)
H = np.vectorize(lambda x: 1/2 if x==0.0 else 1 if x>0.0 else 0.0)

domain = np.linspace(-10,10,1001)
plt.scatter(domain, H(domain), s=.1)
plt.title("Unit step function")
plt.show()

#### Boxcar Function

$$\Pi_{a,b}(x):=H(x-a)-H(x-b)$$
We can approximate any continuous function $f:\mathbb R\rightarrow\mathbb R$ in some closed interval $K=[s,e]$ using boxcar functions:

Given any $\epsilon > 0$, we can find positive integer $n$ such that if $$x_i := s+\frac{e-s}{n}i \qquad \left(0\leq i \leq n\right)$$
and
$$f_n:=\sum_{i=1}^{n}f(x_{i})\Pi_{x_{i-1},x_{i}}$$
then
$$\sup_{x\in K}\|f(x)-f_n(x)\|<\epsilon.$$

Moreover, since

$$
\begin{align*}
f_n &= \sum_{i=1}^{n}f(x_i)\left(H(x-x_{i-1}) - H(x-x_i)\right)\\
&=f(x_1)H(x-x_{0})-\sum_{i=1}^{n-1}\left(f(x_i)-f(x_{i+1})\right)H(x-x_i)-f(x_n)H(x-x_n),
\end{align*}
$$
we have
$$f_n = W_2\cdot H\circ W_1$$
where
$$
\begin{align*}
W_1 &= \left((1,1,\ldots,1)^T, \left(-x_0,-x_1,\ldots,-x_n\right)^T\right)\in\mathbb R^{(n+1)\times 1}\times\mathbb R^{n+1},\\
W_2 &= \left(\left(f(x_1), f(x_2)-f(x_1), f(x_3)-f(x_2),\ldots,f(x_n)-f(x_{n-1}), -f(x_n)\right), 0\right)\in\mathbb R^{1\times (n+1)}\times\mathbb R.
\end{align*}
$$


In [ ]:
boxcar = lambda x, a=-.5, b=.5: H(x-a) - H(x-b)

domain = np.linspace(-2,2,1001)
plt.scatter(domain, boxcar(domain), s=.1)
plt.title("Unit rectangle function")
plt.show()

In [ ]:
f = lambda x: -1 * (x+1) * x * (x-1)

domain = np.linspace(0,1,10001)


for n in [3, 5, 9, 17, 33, 65, 129, 257, 513]:
  d = np.linspace(0,1,n)
  a = d[:-1]
  b = d[1:]
  fn = sum([f(b)*boxcar(domain,a,b) for a,b in list(zip(a,b))])

  plt.figure(figsize=(12,4))

  plt.subplot(1,2,1)
  plt.scatter(domain, f(domain), s=.1, label='$f$')
  plt.scatter(domain, fn, s=.1, label='approx.')
  plt.title(f"An approximation of $f$ (n={n})")
  plt.legend()

  plt.subplot(1,2,2)
  err = f(domain)-fn
  plt.scatter(domain, err, s=.1)
  plt.title(f"Errors ({np.abs(err).max():.5f})")

  plt.show()

* We have approximated $f$ using a number of unit step functions.

By replacing the closed interval $K$ to the product of closed intervals, we can show the general $f:\mathbb R^d\rightarrow\mathbb R^D$ case.



#### Continuous Version of Unit Step Function

In [ ]:
sigmoid = lambda x: 1/(1+np.exp(-x))

domain = np.linspace(-10,10,1001)

plt.scatter(domain, H(domain), s=.1, label='unit step function')
plt.plot([0],[.5]) # waste one color
plt.plot(domain, sigmoid(domain), label='sigmoid')
plt.title("Sigmoid")
plt.legend()

plt.show()

In [ ]:
domain = np.linspace(-2,2,1001)
plt.scatter(domain, H(domain), s=.1, label='unit step function')
plt.plot([0],[.5]) # waste one color
for a in [1, 2, 4, 8, 16, 32]:
  plt.plot(domain, sigmoid(a*domain), label=f'a={a}')
plt.title("Sigmoid(ax)")
plt.legend()
plt.show()

In [ ]:
f = lambda x: -1 * (x+1) * x * (x-1)

sig_boxcar = lambda x, a, b: sigmoid(512*(x-a)) - sigmoid(512*(x-b))

domain = np.linspace(0,1,10001)


for n in [3, 5, 9, 17, 33, 65, 129, 257, 513]:
  d = np.linspace(0,1,n)
  a = d[:-1]
  b = d[1:]
  fn = sum([f(b)*sig_boxcar(domain,a,b) for a,b in list(zip(a,b))])

  plt.figure(figsize=(12,4))

  plt.subplot(1,2,1)
  plt.plot(domain, f(domain), label='$f$')
  plt.plot(domain, fn, label='approx.')
  plt.title(f"An approximation of $f$ (n={n})")
  plt.legend()

  plt.subplot(1,2,2)
  err = f(domain)-fn
  plt.plot(domain, err)
  plt.title(f"Errors ({np.abs(err).max():.5f})")

  plt.show()

* We have approximated $f$ using a number of sigmoid functions.

In [ ]:
relu = lambda x: x*(x>0)

domain = np.linspace(-10,10,1000)

plt.plot(domain, relu(domain))
plt.title("ReLU")
plt.show()

In [ ]:
domain = np.linspace(-2,2,1001)
plt.scatter(domain, H(domain), s=.1, label='unit step function')
plt.plot([0],[.5]) # waste one color
for a in [1, 2, 4, 8, 16, 32]:
  plt.plot(domain, relu(a*domain+0.5) - relu(a*domain-0.5), label=f'a={a}')
plt.title("ReLU(ax+0.5)-ReLU(ax-0.5)")
plt.legend()
plt.show()

In [ ]:
f = lambda x: -1 * (x+1) * x * (x-1)

relu_boxcar = lambda x, a, b: (relu(256*(x-a)+0.5) - relu(256*(x-a)-0.5)) - (relu(256*(x-b)+0.5) - relu(256*(x-b)-0.5))

domain = np.linspace(0,1,10001)


for n in [3, 5, 9, 17, 33, 65, 129, 257, 513]:
  d = np.linspace(0,1,n)
  a = d[:-1]
  b = d[1:]
  fn = sum([f(b)*relu_boxcar(domain,a,b) for a,b in list(zip(a,b))])

  plt.figure(figsize=(12,4))

  plt.subplot(1,2,1)
  plt.plot(domain, f(domain), label='$f$')
  plt.plot(domain, fn, label='approx.')
  plt.title(f"An approximation of $f$ (n={n})")
  plt.legend()

  plt.subplot(1,2,2)
  err = f(domain)-fn
  plt.plot(domain, err)
  plt.title(f"Errors ({np.abs(err).max():.5f})")

  plt.show()

* We have approximated $f$ using a number of ReLU(Rectified Linear Unit) functions.

#### Summary
* continuous function $f:\mathbb R^d\rightarrow\mathbb R^D$
* continuous non-polynomial function $\sigma:\mathbb R\rightarrow\mathbb R$.

Then $f$ can be approximated by a neural network represented by $W_2\circ\sigma\circ W_1$.

## Arbitrary-depth case

[Wikipedia](https://en.wikipedia.org/wiki/Universal_approximation_theorem#Arbitrary-depth_case)

(2017, Zhou Lu et al.)

Let $\mathcal{X}$ be a compact subset of $\mathbb{R}^d$. Let $\sigma:\mathbb{R}\to\mathbb{R}$ be any non-affine continuous function which is continuously differentiable at at-least one point, with non-zero derivative at that point. Let $\mathcal{N}_{d,D:d+D+2}^{\sigma}$ denote the space of feed-forward neural networks with $d$ input neurons, $D$ output neurons, and an arbitrary number of hidden layers each with $d + D + 2$ neurons, such that every hidden neuron has activation function $\sigma$ and every output neuron has the identity as its activation function, with input layer $\phi$, and output layer $\rho$. Then given any $\varepsilon>0$ and any $f\in C(\mathcal{X},\mathbb{R}^D)$, there exists $\hat{f}\in \mathcal{N}_{d,D:d+D+2}^{\sigma}$ such that
$$
\sup_{x \in \mathcal{X}}\,\left\|\hat{f}(x)-f(x)\right\| < \varepsilon.
$$

In other words, $\mathcal{N}$ is dense in $C(\mathcal{X}; \mathbb{R}^D)$ with respect to the uniform topology.


### Sketch
Given a continuous function $\sigma:\mathbb R\rightarrow\mathbb R$

$\sigma$ is continuously differentiable at at-least one point, with non-zero derivative at that point

$\implies$
* $\forall f:\mathbb R^d\rightarrow \mathbb R^D$,
* $\forall$ compact $\mathcal X\subset \mathbb R^d$,
* $\forall \epsilon>0$,

there exists a neural network $\hat f$:
* $d$ input neurons,
* $D$ output neurons,
* an arbitrary number of hidden layers each with $d+D+2$ neurons having activation function $\sigma$,

satisfying
$$\sup_{x\in\mathcal X}\|f(x)-\hat f(x)\|<\epsilon.$$


# Some Lessons from Universal Approximation Theorem

## JAX

* 메뉴 -> 런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU
* 아래의 셀을 실행시켰을때 다음과 같은 결과가 나와야 합니다.
```
Devices: [GpuDevice(id=0, process_index=0)]
```

In [ ]:
from functools import partial

import jax
import jax.numpy as jnp
from jax import random
from jax import jax

import matplotlib.pyplot as plt

print("Devices:", jax.devices())

## Real Valued Function

* $[0,1]$ part of cubic polynomial $$f(x)=-x(x-1)(x+2)$$

* Our data is $$\left\{\left(x_1, f(x_1)+\epsilon_1\right),\ldots, \left(x_{100}, f(x_{100})+\epsilon_{100}\right)\right\}$$
where $\epsilon_i$ are noise.

In [ ]:
plt.style.use('dark_background')

f = lambda x: -x*(x-1)*(x+1)
x = jnp.linspace(0,1,1000).reshape(1000,1)

rng_key = random.PRNGKey(2022)
rng_key, subkey1, subkey2 = random.split(rng_key, 3)

xs = random.uniform(subkey1, shape=(100,1))
# noise = random.normal(subkey2, shape=(100,1)) * 0.01
ys = f(xs) #+ noise

plt.title('Train Dataset')
plt.scatter(xs, ys, label='data')
plt.plot(x, f(x), label='$y=f(x)$', c='red')
plt.legend()
plt.show()

### Model

* Input dim. = 1
* Hidden dim. = 50
* Output dim. = 1

In [ ]:
rng_key, w1_key, w2_key = random.split(rng_key, 3)

params = {
    'W1': random.normal(w1_key, (1,50)),
    'b1': jnp.zeros((1,50)),
    'W2': random.normal(w2_key, (50,1)),
    'b2': jnp.zeros((1,1))
}


@jax.jit
def simple_ann(params, x):
  w1, b1 = params.get('W1'), params.get('b1')
  w2, b2 = params.get('W2'), params.get('b2')

  x = jax.nn.sigmoid(x@w1 + b1)
  return x@w2 + b2

In [ ]:
plt.title('Before Train')

plt.plot(x, f(x),label='$y=f(x)$')
plt.scatter(xs, ys)

plt.plot(x, simple_ann(params, x), label='$y=\hat{f}(x)$')
plt.scatter(xs, simple_ann(params, xs))

plt.legend()
plt.show()

Our model is $\hat y(x;\theta)= \sigma(x\cdot W_1+b_1)\cdot W_2+b_2$.

The loss function is $J(x,y;\theta) = E_\theta (\hat y-y)^2$.

### Gradient Descent with Momentum

$$\theta_{\textrm{new}} = \theta - \eta(\nabla_\theta J)(x,y;\theta)+\alpha\Delta\theta$$

* $\eta$ : learning rate (or step size).
* $\alpha$ : momentum.

In [ ]:
def loss_fn(params, x, y):
  y_hat = simple_ann(params, x)
  return jnp.mean((y_hat - y)**2)

@jax.jit
def update(params, opt_state, x, y):
  loss, grads = jax.value_and_grad(loss_fn)(params, x, y)
  lr, momentum = opt_state['lr'], opt_state['momentum']
  change_old = opt_state['change']
  opt_state['change'] = change = jax.tree_map(lambda g,c: lr*g + momentum*c, grads, change_old)
  params = jax.tree_map(lambda p, c: p - c, params, change)
  return params, opt_state, loss

In [ ]:
opt_state = {
    'lr': 0.1,
    'momentum': 0.8,
    'change': jax.tree_map(lambda g: .0, jax.grad(loss_fn)(params, xs, ys))
    }

print(f'Epoch : 0\tloss:{loss_fn(params, xs, ys):.8f}')
for i in range(200000):
  params, opt_state, loss = update(params, opt_state, xs, ys)
  if i%10000==9999:
    print(f'Epoch : {i+1}\tloss:{loss:.8f}')

In [ ]:
plt.title('After Train')

plt.plot(x, f(x),label='$y=f(x)$')
plt.scatter(xs, ys)

plt.plot(x, simple_ann(params, x), label='$y=\hat{f}(x)$')
plt.scatter(xs, simple_ann(params, xs))

plt.legend()
plt.show()

* Good Job! (Really?)

## Lesson 1

ANN model CANNOT tell about outside of the comapct set - in this case, outside of $[0,1]$.

Note that the universal approximation theorem says:<br>
the approximation bound
$$\sup_{x\in K}\|f(x)-f_\epsilon(x)\|<\epsilon$$
where $K\subset\mathbb{R}^D$ is a compact subset.


In [ ]:
plt.title('Outside of the campact set')
xx = jnp.linspace(-1,1.5,2500)[:,jnp.newaxis]
plt.plot(xx, f(xx),label='$y=f(x)$')
plt.scatter(xs, ys)

plt.plot(xx, simple_ann(params, xx), label='$y=\hat{f}(x)$')
plt.scatter(xs, simple_ann(params, xs))

plt.legend()
plt.show()

## Lesson 2

You should have enough data.

If we have only 4 points of the cubic polynomial, then we get a bad approximation.

In [ ]:
rng_key = random.PRNGKey(2)
rng_key, subkey = random.split(rng_key)
xs2 = random.uniform(subkey, shape=(4,1))
ys2 = f(xs2)

plt.title('Train Dataset')
plt.plot(x, f(x), label='$y=f(x)$')
plt.scatter(xs2, ys2)
plt.legend()
plt.show()

In [ ]:
rng_key, w1_key, w2_key = random.split(rng_key, 3)

params2 = {
    'W1': random.normal(w1_key, (1,50)),
    'b1': jnp.zeros((1,50)),
    'W2': random.normal(w2_key, (50,1)),
    'b2': jnp.zeros((1,1))
}

In [ ]:
plt.title('Before Train')

plt.plot(x, f(x),label='$y=f(x)$')
plt.scatter(xs2, ys2)

plt.plot(x, simple_ann(params2, x), label='$y=\hat{f}(x)$')
plt.scatter(xs2, simple_ann(params2, xs2))

plt.legend()
plt.show()

In [ ]:
opt_state2 = {
    'lr': 0.1,
    'momentum': 0.8,
    'change': jax.tree_map(lambda g: .0, jax.grad(loss_fn)(params2, xs2, ys2))
    }

print(f'Epoch : 0\tloss:{loss_fn(params2, xs2, ys2):.8f}')
for i in range(200000):
  params2, opt_state2, loss = update(params2, opt_state2, xs2, ys2)
  if i%10000==9999:
    print(f'Epoch : {i+1}\tloss:{loss:.8f}')

In [ ]:
plt.title('After Train')

plt.plot(x, f(x),label='$y=f(x)$')
plt.scatter(xs2, ys2)

plt.plot(x, simple_ann(params2, x), label='$y=\hat{f}(x)$')
plt.scatter(xs2, simple_ann(params2, xs2))

plt.legend()
plt.show()

In [ ]:
plt.title('After Train')

plt.plot(xx, f(xx),label='$y=f(x)$')
plt.scatter(xs2, ys2)

plt.plot(xx, simple_ann(params2, xx), label='$y=\hat{f}(x)$')
plt.scatter(xs2, simple_ann(params2, xs2))

plt.legend()
plt.show()

## Lesson 3

***First of all***, you have to check if there exists a exact solution for the problem.

Because we are working on cubic polynomial, 4 points are enough.

Just solve the equation:
$$
\begin{pmatrix}
x_1^3 & x_1^2 & x_1 & 1\\
x_2^3 & x_2^2 & x_2 & 1\\
x_3^3 & x_3^2 & x_3 & 1\\
x_4^3 & x_4^2 & x_4 & 1\\
\end{pmatrix}
\begin{pmatrix}
a\\ b\\ c\\ d\\
\end{pmatrix}
=
\begin{pmatrix}
y_1 \\y_2 \\y_3 \\y_4 \\
\end{pmatrix}
$$

In [ ]:
A = jnp.concatenate([xs2**i for i in [3,2,1,0]], axis=1)
A

In [ ]:
a,b,c,d = jnp.linalg.inv(A) @ ys2

g = lambda x: a*x**3 + b*x**2 + c*x + d

print(f'{a.item():.4f}x^3 + {b.item():.4f}x^2 + {c.item():.4f}x + {d.item():.4f}')

plt.title('Exact solution')
plt.plot(xx, f(xx),label='$y=f(x)$')
plt.scatter(xs2, ys2)

plt.plot(xx, g(xx), label='$y=g(x)$')
plt.scatter(xs2, g(xs2))

plt.legend()
plt.show()

These lessons are just the first stpe of your journey to data science.

By just reading the statement of the universal approximation theorem,
you can figure out many strenghs and weaknesses of ANN.

Good luck.
